# Installing the OpenBB Platform in Google Colab

This notebook will install the OpenBB Platform, fetch some data and prepare it for display as a bar chart.

Sign up for a free account here: https://my.openbb.co

In [ ]:
# Install the OpenBB Platform with all available extensions.
# Messages indicating package version conflicts at the end of installation can be safely ignored.

!pip install openbb[all]

# There is also a nightly distribution available, openbb-nightly

In [98]:
# Before running this cell, restart the runtime by selecting, "Restart runtime", from the "Runtime" menu.

# Import statements - for many scenarios, the only import needed will be `from openbb import obb`
from typing import Literal
from IPython.display import display
from IPython.display import clear_output
import ipywidgets as widgets
import pandas as pd
import pandas_ta as ta
from datetime import datetime
from plotly import graph_objects as go

from openbb import obb

In [3]:
# Login to OpenBB Hub to retrieve stored API keys.
# https://my.openbb.co/app/platform/pat
# https://my.openbb.co/app/platform/api-keys

obb.account.login(pat="replace with your PAT")

# This is not required

In [4]:
# Verify that the credentials from Hub were loaded successfully.

obb.user.credentials

Credentials

alpha_vantage_api_key: **********
benzinga_api_key: None
biztoc_api_key: None
fmp_api_key: **********
fred_api_key: **********
intrinio_api_key: **********
nasdaq_api_key: **********
polygon_api_key: **********
tiingo_token: None
tradingeconomics_api_key: None

In [67]:
# Set the output preference, if desired. The examples below use Pandas DataFrames.

obb.user.preferences.output_type = "dataframe"

In [88]:
# Get Some Data
symbol = "SPY"

options = obb.derivatives.options.chains(symbol, provider="cboe")

options

,contract_symbol,expiration,strike,option_type,volume,open,open_interest,high,low,implied_volatility,...,last_trade_price,tick,prev_close,change,change_percent,rho,last_trade_timestamp,dte,bid,ask
0,SPY231130C00387000,2023-11-30,387.0,call,7.0,67.60,5.0,67.60,66.55,0.0000,...,66.55,down,67.69,-1.135,-1.68,0.0003,2023-11-30 12:27:59,-1,67.77,70.67
1,SPY231130P00387000,2023-11-30,387.0,put,2.0,0.01,290.0,0.01,0.01,0.0000,...,0.01,no_change,0.00,0.005,100.00,0.0000,2023-11-30 09:30:12,-1,0.00,0.01
2,SPY231130C00388000,2023-11-30,388.0,call,1.0,66.47,0.0,66.47,66.47,0.0000,...,66.47,down,66.68,-0.210,-0.31,0.0003,2023-11-30 09:42:00,-1,67.56,68.77
3,SPY231130P00388000,2023-11-30,388.0,put,1.0,0.01,1.0,0.01,0.01,0.0000,...,0.01,no_change,0.00,0.005,100.00,0.0000,2023-11-30 10:16:04,-1,0.00,0.01
4,SPY231130C00389000,2023-11-30,389.0,call,0.0,0.00,0.0,0.00,0.00,0.0000,...,0.00,no_change,65.69,0.000,0.00,0.0003,NaT,-1,65.77,68.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8215,SPY260116P00670000,2026-01-16,670.0,put,0.0,0.00,0.0,0.00,0.00,0.2367,...,216.04,down,215.44,0.000,0.00,-0.3427,2023-11-20 12:31:47,777,211.56,216.50
8216,SPY260116C00675000,2026-01-16,675.0,call,0.0,0.00,0.0,0.00,0.00,0.1387,...,0.00,no_change,2.50,0.000,0.00,0.4385,NaT,777,0.00,5.00
8217,SPY260116P00675000,2026-01-16,675.0,put,0.0,0.00,0.0,0.00,0.00,0.2385,...,0.00,no_change,220.50,0.000,0.00,-0.3453,NaT,777,216.50,221.50
8218,SPY260116C00680000,2026-01-16,680.0,call,0.0,0.00,0.0,0.00,0.00,0.1411,...,0.00,no_change,2.50,0.000,0.00,0.4078,NaT,777,0.00,5.00


In [101]:
# Prepare A View - Volume and Open Interest by Expiration or Strike

def filter_options_data(options, by: Literal["expiration", "strike"] = "expiration"):
    data = pd.DataFrame()
    data["Total Open Interest"] = options.groupby(by)["open_interest"].sum()
    data["Call Open Interest"] = options[options["option_type"] == "call"].groupby(by)["open_interest"].sum()
    data["Put Open Interest"] = options[options["option_type"] == "put"].groupby(by)["open_interest"].sum()
    data["Total Volume"] = options.groupby(by)["volume"].sum()
    data["Call Volume"] = options[options["option_type"] == "call"].groupby(by)["volume"].sum()
    data["Put Volume"] = options[options["option_type"] == "put"].groupby(by)["volume"].sum()

    return data

data = filter_options_data(options, "strike")

data

,Total Open Interest,Call Open Interest,Put Open Interest,Total Volume,Call Volume,Put Volume
strike,,,,,,
120.0,22821.0,74.0,22747.0,78.0,0.0,78.0
130.0,1506.0,19.0,1487.0,61.0,0.0,61.0
140.0,488.0,2.0,486.0,1.0,0.0,1.0
150.0,72684.0,2072.0,70612.0,65.0,4.0,61.0
155.0,12298.0,18.0,12280.0,21.0,1.0,20.0
...,...,...,...,...,...,...
700.0,24945.0,24835.0,110.0,2.0,2.0,0.0
705.0,478.0,478.0,0.0,0.0,0.0,0.0
710.0,1511.0,1511.0,0.0,3.0,1.0,2.0


In [93]:
# Do not run this cell if you are following the example above.

# In this scenario, "data" could be anything that would be displayed as a bar chart. Alternatively, it could be company fundamentals data.

# Note: This requires a valid FMP API key


symbol="AAPL"

data = obb.equity.fundamental.ratios(symbol, limit = 100, period="quarter", provider="fmp")

data.index = data.index.strftime("%Y-%m-%d")

data

,symbol,period,current_ratio,quick_ratio,cash_ratio,days_of_sales_outstanding,days_of_inventory_outstanding,operating_cycle,days_of_payables_outstanding,cash_conversion_cycle,...,price_earnings_ratio,price_to_free_cash_flows_ratio,price_to_operating_cash_flows_ratio,price_cash_flow_ratio,price_earnings_to_growth_ratio,price_sales_ratio,dividend_yield,enterprise_value_multiple,price_fair_value,calendarYear
date,,,,,,,,,,,,,,,,,,,,,
1998-12-26,AAPL,Q1,2.605795,2.352426,0.822776,48.052632,1.832248,49.884879,48.004886,1.879993,...,8.732568,24.355053,23.808976,23.808976,0.203760,3.104913,0.000000,28.650009,2.760999,1999
1999-03-27,AAPL,Q2,2.650259,2.413212,0.879534,47.294118,1.437445,48.731562,63.167702,-14.436140,...,8.397059,18.137648,16.856551,16.856551,-0.699755,2.963668,0.000000,62.597153,2.084787,1999
1999-06-26,AAPL,Q3,2.792723,2.600390,1.152047,51.758665,0.557029,52.315694,63.023873,-10.708179,...,7.484225,75.027047,69.058986,69.058986,0.178003,3.900636,0.000000,209.281398,2.046192,1999
1999-09-25,AAPL,Q4,2.766301,2.522272,0.856036,45.875749,1.890756,47.766505,76.764706,-28.998201,...,23.552897,51.769733,47.970120,47.970120,-0.467319,7.827460,0.000000,70.913430,3.369035,1999
2000-01-01,AAPL,Q1,2.498219,2.316539,0.807125,34.263764,0.777650,35.041414,60.864055,-25.822641,...,22.618570,49.423264,44.388186,44.388186,0.350588,7.066493,0.000000,58.063853,3.750973,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-24,AAPL,Q4,0.879356,0.709408,0.153563,60.833315,8.551997,69.385312,110.859542,-41.474230,...,29.094281,115.723695,99.948206,99.948206,3.879237,26.750498,0.001536,90.344514,47.589406,2022
2022-12-31,AAPL,Q1,0.938020,0.768724,0.149578,41.622138,9.185598,50.807736,78.007542,-27.199806,...,17.208993,68.334817,60.724643,60.724643,0.369993,17.625873,0.001825,55.366190,36.401387,2023
2023-04-01,AAPL,Q2,0.940354,0.764281,0.205597,34.068392,12.738933,46.807325,73.118615,-26.311290,...,26.938138,101.516990,91.152020,91.152020,-1.414252,27.450564,0.001402,86.117046,41.882005,2023


In [102]:
# Create a widget for selecting the data to display.

clear_output(wait = False)

data_choices = data.columns.tolist()
data_selection = widgets.Dropdown(
    options = data_choices,
    value = None,
)
output = widgets.Output()


def generate_figure(data, data_choice):
    data = data[data[data_choice].notnull()]
    fig = go.Figure()
    fig.add_bar(
        y = data[data_choice][data[data_choice] > 0].values,
        x = data[data_choice][data[data_choice] > 0].index,
        name = data_choice,
        marker = dict(color = "blue"),
    )
    fig.add_bar(
        y = data[data_choice][data[data_choice] < 0].values,
        x = data[data_choice][data[data_choice] < 0].index,
        name = data_choice,
        marker = dict(color = "red")
    )
    fig.update_xaxes(type="category")
    fig.update_traces(width=0.98, selector=dict(type="bar"))
    fig.update_layout(
        showlegend=False,
        width=1400,
        height=600,
        title = dict(
            text=f"{symbol} {data_choice.replace('_', ' ').title()}",
            xanchor = "center",
            x = 0.5,
            font = dict(size = 20)
        ),
        barmode="overlay",
        bargap=0,
        bargroupgap=0,
        yaxis=dict(
            ticklen=0,
            showgrid=True,
            tickfont=dict(size=14),
        ),
        xaxis=dict(
            showgrid=False,
            autorange=True,
            tickangle=90,
            tickfont=dict(size=11),
        ),
    )
    return fig

def on_value_change(change):
    clear_output(wait = True)
    display(data_selection)
    with output:
        data_selection.value

data_selection.observe(on_value_change, names="value")
display(data_selection)

# Select from the drop-down menu below.

Dropdown(options=('Total Open Interest', 'Call Open Interest', 'Put Open Interest', 'Total Volume', 'Call Volu…

In [106]:
# Play this cell to display the choice

if data_selection.value is not None:

    generate_figure(data, data_selection.value).show()